<a href="https://colab.research.google.com/github/arvartho/MovieGenreClassifier/blob/master/movies_binary_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy==2.2.3

In [0]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_selection import SelectPercentile, chi2
import io

#from google.colab import files
#uploaded = files.upload()

# Loading the input files
trainData = pd.read_csv(io.StringIO(uploaded['movieDB_clean_train.csv'].decode('utf-8')))
testData = pd.read_csv(io.StringIO(uploaded['movieDB_clean_test.csv'].decode('utf-8')))


In [27]:
trainData

,title,plot,Οικογενειακή,Δραμεντί,Περιπέτεια,Κωμωδία,Δράσης,Αισθηματική,Δραματική,Θρίλερ,Δραματικό Θρίλερ,Φαντασίας,Επιστημονικής Φαντασίας,Εποχής
0,'Επίθεση στο Σταθμό 13',Αυτή η παραμονή Πρωτοχρονιάς θα είναι διαφορετ...,0,0,0,0,1,0,0,1,0,0,0,0
1,'Περσόνα',"Η Ελβίρα, διάσημη ηθοποιός, καταρρέει κατά τη ...",0,0,0,0,0,0,1,0,1,0,0,0
2,'Ο Ατσάλινος Γίγαντας',"1957, καταμεσής του Ψυχρού Πολέμου. Το μόνο πο...",1,1,1,0,1,0,0,0,0,0,1,0
3,'Οι Διώκτες του Εγκλήματος',"Λος Άντζελες, 1949. Ο αδίστακτος μαφιόζος Μίκι...",0,0,0,0,1,0,0,1,0,0,0,1
4,'Το Τελευταίο Τρένο για τη Γιούμα',"Ο Μπεν Γουέιντ, αρχηγός μιας συμμορίας παρανόμ...",0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928,'Το Κοινόβιο',Δεκαετία του 1970. Ο Έρικ και η Άννα είναι ένα...,0,1,0,0,0,0,1,0,0,0,0,1
2929,'Το Θήραμα',"Είναι απλό. Ο Μάντεκ είναι πλούσιος, ενώ ο Μπε...",0,0,1,0,0,0,0,1,0,0,0,0
2930,'Ανεκπλήρωτος Γάμος',"Βρετανία, Ιούλιος 1962. Ο Έντουαρντ και η Φλόρ...",0,0,0,0,0,1,1,0,0,0,0,1
2931,'Πέμπτη & 12',Ο Ευτύχης είναι πραγματικά άτυχος! Δεν του κάθ...,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
def removeStopwords(df,column):
    from spacy.lang.el import Greek
    from spacy.lang.el.stop_words import STOP_WORDS
    pd.options.mode.chained_assignment = None

    # Load Greek tokenizer, tagger, parser, NER and word vectors
    nlp = Greek()
    # "nlp" Object is used to create documents with linguistic annotations.
    sentences = []
    
    for i, data in df.iterrows():
        sent = nlp(data[column])

        # Create list of word tokens
        token_list = []
        for token in sent:
            dir(token)
            token_list.append(token.lemma_)

        # Create list of word tokens after removing stopwords
        filtered_sentence =[]

        for word in token_list:
            lexeme = nlp.vocab[str(word)]
            if lexeme.is_stop == False:
                filtered_sentence.append(str(word).lower())
        sentence = ' '.join(map(str, filtered_sentence))
        sentences.append(sentence)
    col_name = "new_" + str(column)
    df[col_name] = sentences
    del df[column]

In [0]:
# Lemmatize and remove stopwords in title and plot using function in utilities functions and parse the columns with spacy libraries
removeStopwords(testData,'title')
removeStopwords(testData,'plot')
removeStopwords(trainData,'title')
removeStopwords(trainData,'plot')


In [31]:
# updated title and plot
trainData

,Οικογενειακή,Δραμεντί,Περιπέτεια,Κωμωδία,Δράσης,Αισθηματική,Δραματική,Θρίλερ,Δραματικό Θρίλερ,Φαντασίας,Επιστημονικής Φαντασίας,Εποχής,new_title,new_plot
0,0,0,0,0,1,0,0,1,0,0,0,0,' επίθεση σταθμό 13',παραμονή πρωτοχρονιάς διαφορετική αξιωματικό α...
1,0,0,0,0,0,0,1,0,1,0,0,0,'περσόνα',"ελβίρα , διάσημη ηθοποιός , καταρρέει διάρκεια..."
2,1,1,1,0,1,0,0,0,0,0,1,0,' ατσάλινος γίγαντας ',"1957 , καταμεσής ψυχρού πολέμου . επιθυμεί 9χρ..."
3,0,0,0,0,1,0,0,1,0,0,0,1,' διώκτες εγκλήματος ',"λος άντζελες , 1949 . αδίστακτος μαφιόζος μίκι..."
4,0,0,0,0,1,0,0,0,0,0,0,1,' τρένο γιούμα ',"μπεν γουέιντ , αρχηγός συμμορίας παρανόμων , σ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928,0,1,0,0,0,0,1,0,0,0,0,1,' κοινόβιο ',δεκαετία 1970 . έρικ άννα ζευγάρι ακαδημαϊκών ...
2929,0,0,1,0,0,0,0,1,0,0,0,0,' θήραμα ',"απλό . μάντεκ πλούσιος , μπεν . προσλαμβάνει δ..."
2930,0,0,0,0,0,1,1,0,0,0,0,1,' ανεκπλήρωτος γάμος ',"βρετανία , ιούλιος 1962 . έντουαρντ φλόρενς , ..."
2931,0,0,0,1,0,0,0,0,0,0,0,0,' πέμπτη & 12',ευτύχης πραγματικά άτυχος ! κάθεται . πάει καλ...


In [32]:
# split train and test sets. The test set contains 20% of data and training set 80%

train_X, train_y = trainData['new_plot'], trainData.drop(['new_title', 'new_plot'], axis=1)
test_X, test_y = testData['new_plot'], testData.drop(['new_title', 'new_plot'], axis=1)

categories = train_y.columns
# Categories that will be used for multi label
categories

Index(['Οικογενειακή', 'Δραμεντί', 'Περιπέτεια', 'Κωμωδία', 'Δράσης',
       'Αισθηματική', 'Δραματική', 'Θρίλερ', 'Δραματικό Θρίλερ', 'Φαντασίας',
       'Επιστημονικής Φαντασίας', 'Εποχής'],
      dtype='object')

""" Binary Relevance:

The Baseline approach, called the binary relevance method, amounts to independently training one binary classifier for each label.
Given an unseen sample, the combined model then predicts all labels for this sample for which the respective classifiers predict a positive result.
We use the inbuilt sklearn OneVsRestClassifier function to achieve this multi-label classification.
OnevsRestClassifier is commonly used for multi-class classification but it can also be used for multi-label classification.

Probability threshold used for classifying each genres is based on the frequency of its occurence

"""

In [0]:
def getProbThresh(mydata, threshSel=1, threshOffset=0):
    '''
    The probability threshold to be used for making classification decisions
    threshSel = 1 : Default 0.5 probability threshold
    threshSel = 2 : max(0.5, Fraction of genre occurence + threshOffset)
    '''
    numGenres = mydata.shape[1]
    probThresh = []
    if threshSel == 1:
        probThresh = [0.5] * numGenres
    elif threshSel == 2:
        sumGenre = mydata.sum()
        probThresh = (sumGenre / mydata.shape[0] + threshOffset).clip(upper=0.5)
    return probThresh

In [36]:
probThresh = getProbThresh(trainData[categories], threshSel=1)
probThresh

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

In [0]:
def multiLabelPredict(clf, X_test, probThresh, categories):
    '''
    Multi-label prediction based on probability threshold.
    Prediction is made based on Binary Relevance where each genre has a separate classifier
    '''
    categoryColumns = categories
    
    y_pred = pd.DataFrame(columns=categoryColumns)
    
    prob = clf.predict_proba(X_test)
    for idx, col in enumerate(categoryColumns):
        y_pred[col] = prob[:,idx] > probThresh[idx]
    prob = pd.DataFrame(prob, columns=categoryColumns)
    return prob, y_pred

In [0]:
def printScores(methodName, y_test, y_pred):
    '''
    Printing on StdOut precision, recall, accuracy and F1-Measure
    '''
    result = pd.DataFrame(columns=['Precision', 'Recall', 'F1-Score', 'Support'])
    tp, fp, fn, total = 0, 0, 0, 0
    if not isinstance(y_pred, pd.DataFrame):
        y_pred = pd.DataFrame(y_pred, columns=y_test.columns)
    for col in y_test.columns:
        support = y_test[col].sum()
        tp = ((y_test[col]==1) & (y_pred[col]==1)).sum()
        fp = ((y_test[col]==0) & (y_pred[col]==1)).sum()
        fn = ((y_test[col]==1) & (y_pred[col]==0)).sum()
        
        precision = 0 if (tp+fp==0) else tp/(tp+fp)
        recall = 0 if (tp+fn==0) else tp/(tp+fn)
        f1_score = 0 if (precision==0 and recall==0) else 2*precision*recall/(precision+recall)
        
        result.loc[col] = [precision, recall, f1_score, support]
    
    avg_precision = (result['Precision']*result['Support']).sum()/result['Support'].sum()
    avg_recall = (result['Recall']*result['Support']).sum()/result['Support'].sum()
    avg_f1_score = (result['F1-Score']*result['Support']).sum()/result['Support'].sum()
    result.loc['Avg/Total'] = [avg_precision, avg_recall, avg_f1_score, result['Support'].sum()]
    
    print(str(round(result, 2)))

In [39]:
# Bayes
print("")
print("Starting Bayes classifiers...")
print("")

pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(class_prior=None)))
            ])

parameters = [
            {
            'tfidf__max_df': (0.25, 0.5, 0.75),
            'tfidf__min_df': (1,2),
            'tfidf__ngram_range': [(1, 1), (1, 2),(1, 3)],
            'clf__estimator__alpha': (0.001, 0.01, 0.4, 0.8, 1),
            'clf__estimator__fit_prior': (True,False)
            }
         ]

grid_search_cv = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1)
grid_search_cv.fit(train_X, train_y)

best_clf = grid_search_cv.best_estimator_
y_pred = best_clf.predict(test_X)
printScores('BAYES', test_y, y_pred)


Starting Bayes classifiers...

                         Precision  Recall  F1-Score  Support
Οικογενειακή                  0.50    0.24      0.32     51.0
Δραμεντί                      0.42    0.12      0.19    158.0
Περιπέτεια                    0.63    0.50      0.56    117.0
Κωμωδία                       0.41    0.34      0.37     83.0
Δράσης                        0.64    0.59      0.61     87.0
Αισθηματική                   0.48    0.24      0.32    131.0
Δραματική                     0.53    0.43      0.47    223.0
Θρίλερ                        0.54    0.32      0.41    145.0
Δραματικό Θρίλερ              0.38    0.04      0.07    162.0
Φαντασίας                     0.83    0.32      0.46    125.0
Επιστημονικής Φαντασίας       0.76    0.35      0.48     96.0
Εποχής                        0.72    0.40      0.52    145.0
Avg/Total                     0.56    0.32      0.39   1523.0


In [41]:
# Linear SVM
print("")
print("Starting Linear SVM classifiers...")
print("")
from sklearn.svm import LinearSVC

pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=-1)),
            ])

# In order to obtain the parameters we used pipeline.get_params().keys()
parameters = [
            {
            'tfidf__max_df': (0.25, 0.5, 0.75),
            'tfidf__min_df': (1,2),
            'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
            'clf__estimator__class_weight': ['balanced'],
            }
         ]

grid_search_cv = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1)
grid_search_cv.fit(train_X, train_y)

# measuring performance on test set
best_clf = grid_search_cv.best_estimator_
y_pred = best_clf.predict(test_X)
printScores('Linear SVM', test_y, y_pred)


Starting Linear SVM classifiers...

                         Precision  Recall  F1-Score  Support
Οικογενειακή                  0.45    0.25      0.32     51.0
Δραμεντί                      0.44    0.17      0.25    158.0
Περιπέτεια                    0.61    0.56      0.58    117.0
Κωμωδία                       0.43    0.40      0.41     83.0
Δράσης                        0.60    0.56      0.58     87.0
Αισθηματική                   0.50    0.39      0.44    131.0
Δραματική                     0.54    0.52      0.53    223.0
Θρίλερ                        0.57    0.32      0.41    145.0
Δραματικό Θρίλερ              0.46    0.07      0.13    162.0
Φαντασίας                     0.83    0.30      0.44    125.0
Επιστημονικής Φαντασίας       0.82    0.38      0.51     96.0
Εποχής                        0.70    0.46      0.55    145.0
Avg/Total                     0.58    0.36      0.43   1523.0


In [42]:
# RBF SVM
from sklearn.svm import SVC
print("")
print("Starting RBF SVM classifiers...")
print("")
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', OneVsRestClassifier(SVC(gamma=2, C=1,probability=True), n_jobs=-1)),
            ])
parameters = {
            'tfidf__max_df': (0.25, 0.5, 0.75),
            'tfidf__min_df': (1,2),
            'tfidf__ngram_range': [(1, 1), (1, 3)],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1)
grid_search_cv.fit(train_X, train_y)

# measuring performance on test set
best_clf = grid_search_cv.best_estimator_
prob, y_pred = multiLabelPredict(best_clf, test_X, probThresh, categories)
printScores('RBF SVM', test_y, y_pred)



Starting RBF SVM classifiers...



/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


                         Precision  Recall  F1-Score  Support
Οικογενειακή                  0.33    0.39      0.36     51.0
Δραμεντί                      0.41    0.04      0.08    158.0
Περιπέτεια                    0.56    0.58      0.57    117.0
Κωμωδία                       0.44    0.42      0.43     83.0
Δράσης                        0.63    0.68      0.66     87.0
Αισθηματική                   0.51    0.45      0.48    131.0
Δραματική                     0.54    0.52      0.53    223.0
Θρίλερ                        0.66    0.34      0.45    145.0
Δραματικό Θρίλερ              1.00    0.01      0.02    162.0
Φαντασίας                     0.78    0.34      0.48    125.0
Επιστημονικής Φαντασίας       0.78    0.45      0.57     96.0
Εποχής                        0.64    0.51      0.57    145.0
Avg/Total                     0.62    0.38      0.42   1523.0


In [43]:
# Neural Net
from sklearn.neural_network import MLPClassifier
print("")
print("Starting Neural Net classifiers...")
print("")
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('select_features', SelectPercentile(chi2, percentile = 40)),
            # NN Clasifier will need much resources so it will not be optimal to try different parameters
            ('clf', OneVsRestClassifier(MLPClassifier(solver='lbfgs', tol=0.00001, activation='relu', hidden_layer_sizes=(25,25,25), max_iter = 500), n_jobs=6)),
            ])
parameters = {
            'tfidf__max_df': (0.25, 0.5, 0.75),
            'tfidf__min_df': (1,2),
            'tfidf__ngram_range': [(1, 1), (1, 3)],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1)
grid_search_cv.fit(train_X, train_y)

# measuring performance on test set
best_clf = grid_search_cv.best_estimator_
prob, y_pred = multiLabelPredict(best_clf, test_X, probThresh, categories)
printScores('Neural Net', test_y, y_pred)


Starting Neural Net classifiers...



/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


                         Precision  Recall  F1-Score  Support
Οικογενειακή                  0.44    0.24      0.31     51.0
Δραμεντί                      0.37    0.18      0.24    158.0
Περιπέτεια                    0.63    0.53      0.57    117.0
Κωμωδία                       0.29    0.35      0.32     83.0
Δράσης                        0.67    0.53      0.59     87.0
Αισθηματική                   0.00    0.00      0.00    131.0
Δραματική                     0.51    0.41      0.46    223.0
Θρίλερ                        0.69    0.24      0.36    145.0
Δραματικό Θρίλερ              0.00    0.00      0.00    162.0
Φαντασίας                     0.77    0.34      0.48    125.0
Επιστημονικής Φαντασίας       0.80    0.41      0.54     96.0
Εποχής                        0.66    0.48      0.55    145.0
Avg/Total                     0.47    0.30      0.36   1523.0
